In [3]:
import numpy as np
import scipy.interpolate
from scipy import interpolate
import math
import matplotlib.pyplot as plt
import subprocess
from astropy.io import fits
from subprocess import Popen, PIPE
from mpl_toolkits.mplot3d import axes3d
from numpy import cos,sin
import h5py
import json

%matplotlib inline

In [4]:
spin=0.5;
d1=0;

#Observation angle, will be used to transform hbar to hTT 
PHI=0/180*np.pi  #azimuthal angle, in rad
THETA=45/180*np.pi  #latitude angle, in rad

r0=20
E=0.968693
Lz=4.077016


In [5]:
#calculate GW from geodesic, see file "Gwave.ipynb"
Nread=0

f=h5py.File("test.hdf5","w")
#geo8RK4
for r0 in np.arange(5,20,0.5):
    for E in np.arange(0.9,1,0.01):

        print(r0,E)
        try:
            index, tau,t,r,th,phi,ut,ur,uth,uphi,F_t,F_r,F_th,F_phi=np.loadtxt \
('C:\\Users\\HP\\Documents\\Visual Studio 2015\\Projects\\RK4geo8_dt\\RK4geo8_dt\
\\trace_spin%.6f_d1%.6f_r0%.6f_E%.6f.dat'%(spin,d1,r0,E),unpack=True)
        except:
            print('trace_spin%.6f_d1%.6f_r0%.6f_E%.6f.dat  does not exist'%(spin,d1,r0,E))
        
        if index[-1]<5000:
            print("skip unstable orbit")
            continue
        else:
            Nread=Nread+1
        print(Nread)
        #qseudo_flat spacetime
        x=[];
        y=[];
        z=[];
        t_tau_dot=[]
        z_tau_dot=[]
        y_tau_dot=[]
        x_tau_dot=[]
        z_t_dot=[]
        y_t_dot=[]
        x_t_dot=[]
        vr_tau_dot=[]
        vth_tau_dot=[]
        vphi_tau_dot=[]
        vx_tau_dot=[]
        vy_tau_dot=[]
        vz_tau_dot=[]
        x_t_2dot=[]
        y_t_2dot=[]
        z_t_2dot=[]


        for i in np.arange(index.size):
            x.append(r[i]*np.sin(th[i])*np.cos(phi[i]));
            y.append(r[i]*np.sin(th[i])*np.sin(phi[i]));
            z.append(r[i]*np.cos(th[i]));
            t_tau_dot.append(ut[i])
            x_tau_dot.append(ur[i]*np.sin(th[i])*np.cos(phi[i]) + r[i]*np.cos(th[i])*np.cos(phi[i])*uth[i] - r[i]*np.sin(th[i])*np.sin(phi[i])*uphi[i] )
            y_tau_dot.append(ur[i]*np.sin(th[i])*np.sin(phi[i]) + r[i]*np.cos(th[i])*np.sin(phi[i])*uth[i] + r[i]*np.sin(th[i])*np.cos(phi[i])*uphi[i] )
            z_tau_dot.append(ur[i]*np.cos(th[i]) - r[i]*np.sin(th[i])*uth[i])
            x_t_dot.append(x_tau_dot[i]/t_tau_dot[i])
            y_t_dot.append(y_tau_dot[i]/t_tau_dot[i])
            z_t_dot.append(z_tau_dot[i]/t_tau_dot[i])

            vr_tau_dot.append( (F_r[i]*t_tau_dot[i]-ur[i]*F_t[i])/t_tau_dot[i]/t_tau_dot[i] )
            vth_tau_dot.append( (F_th[i]*t_tau_dot[i]-uth[i]*F_t[i])/t_tau_dot[i]/t_tau_dot[i] )
            vphi_tau_dot.append( (F_phi[i]*t_tau_dot[i]-uphi[i]*F_t[i])/t_tau_dot[i]/t_tau_dot[i] )

            vx_tau_dot.append( vr_tau_dot[i]*np.sin(th[i])*np.cos(phi[i]) + ur[i]/ut[i]*np.cos(th[i])*np.cos(phi[i])*uth[i] - ur[i]/ut[i]*np.sin(th[i])*np.sin(phi[i])*uphi[i]\
                 + ur[i]*cos(th[i])*cos(phi[i])*uth[i]/ut[i] - r[i]*sin(th[i])*cos(phi[i])*uth[i]/ut[i]*uth[i] -r[i]*cos(th[i])*sin(phi[i])*uth[i]/ut[i]*uphi[i] +r[i]*cos(th[i])*cos(phi[i])*vth_tau_dot[i]  \
                 - ur[i]*sin(th[i])*sin(phi[i])*uphi[i]/ut[i] - r[i]*cos(th[i])*sin(phi[i])*uphi[i]/ut[i]*uth[i] - r[i]*sin(th[i])*cos(phi[i])*uphi[i]/ut[i]*uphi[i] - r[i]*sin(th[i])*sin(phi[i])*vphi_tau_dot[i])

            vy_tau_dot.append( vr_tau_dot[i]*np.sin(th[i])*np.sin(phi[i]) + ur[i]/ut[i]*np.cos(th[i])*np.sin(phi[i])*uth[i] + ur[i]/ut[i]*np.sin(th[i])*np.cos(phi[i])*uphi[i]\
                 + ur[i]*cos(th[i])*sin(phi[i])*uth[i]/ut[i] - r[i]*sin(th[i])*sin(phi[i])*uth[i]/ut[i]*uth[i] +r[i]*cos(th[i])*cos(phi[i])*uth[i]/ut[i]*uphi[i] +r[i]*cos(th[i])*sin(phi[i])*vth_tau_dot[i]  \
                 + ur[i]*sin(th[i])*cos(phi[i])*uphi[i]/ut[i] + r[i]*cos(th[i])*cos(phi[i])*uphi[i]/ut[i]*uth[i] - r[i]*sin(th[i])*sin(phi[i])*uphi[i]/ut[i]*uphi[i] + r[i]*sin(th[i])*cos(phi[i])*vphi_tau_dot[i])

            vz_tau_dot.append( vr_tau_dot[i]*cos(th[i]) -ur[i]/ut[i]*sin(th[i])*uth[i] \
                             -ur[i]*sin(th[i])*uth[i]/ut[i] -r[i]*cos(th[i])*uth[i]/ut[i]*uth[i] - r[i]*sin(th[i])*vth_tau_dot[i] )

            x_t_2dot.append(vx_tau_dot[i]/ut[i])
            y_t_2dot.append(vy_tau_dot[i]/ut[i])
            z_t_2dot.append(vz_tau_dot[i]/ut[i])

        print("derivative finished")
        #四极矩算法，在trace-reversed gauge的metric

        hbar_xx=[]
        hbar_yy=[]
        hbar_zz=[]
        hbar_xy=[]
        hbar_yz=[]
        hbar_xz=[]
        for i in np.arange(index.size):
            hbar_xx.append(4*(x_t_dot[i]*x_t_dot[i]+x[i]*x_t_2dot[i]))
            hbar_yy.append(4*(y_t_dot[i]*y_t_dot[i]+y[i]*y_t_2dot[i]))
            hbar_zz.append(4*(z_t_dot[i]*z_t_dot[i]+z[i]*z_t_2dot[i]))
            hbar_xy.append(2*(y[i]*x_t_2dot[i]+y_t_2dot[i]*x[i]+2*y_t_dot[i]*x_t_dot[i]))
            hbar_yz.append(2*(y[i]*z_t_2dot[i]+y_t_2dot[i]*z[i]+2*y_t_dot[i]*z_t_dot[i]))
            hbar_xz.append(2*(z[i]*x_t_2dot[i]+z_t_2dot[i]*x[i]+2*z_t_dot[i]*x_t_dot[i]))

        #由trace-reversed gauge转换到transverse traceless gauge


        hTT_TT=[]
        hTT_PP=[]
        hTT_TP=[]
        hTT_plus=[]
        hTT_cross=[]

        for i in np.arange(index.size):


            hTT_TT.append( np.cos(THETA)*np.cos(THETA)* (hbar_xx[i]*np.cos(PHI)*np.cos(PHI) + hbar_xy[i]*np.sin(2*PHI) + hbar_yy[i]*np.sin(PHI)*np.sin(PHI) )  +  hbar_zz[i]*np.sin(THETA)*np.sin(THETA)  -  np.sin(2*THETA)* (hbar_xz[i]*np.cos(PHI)+hbar_yz[i]*np.sin(PHI))  )
            hTT_TP.append( np.cos(THETA)* (-0.5*hbar_xx[i]*np.sin(2*PHI) + hbar_xy[i]*np.cos(2*PHI) + 0.5*hbar_yy[i]*np.sin(2*PHI))  +  np.sin(THETA)* (hbar_xz[i]*np.sin(PHI)-hbar_yz[i]*np.cos(PHI)) )
            hTT_PP.append( hbar_xx[i]*np.sin(PHI)*np.sin(PHI)  -  hbar_xy[i]*np.sin(2*PHI)  +  hbar_yy[i]*np.cos(PHI)*np.cos(PHI) )
            hTT_plus.append(0.5*(hTT_TT[i]-hTT_PP[i]))
            hTT_cross.append(hTT_TP[i])




        filename="/Kerr/tmp%d"%(Nread)
        f[filename]=np.array(hTT_plus)+np.array(hTT_cross)*1j
        print(f[filename].dtype)
        print(filename)
        f[filename].attrs['r0']=r0
        f[filename].attrs['E']=E
        f[filename].attrs['spin']=spin
        f[filename].attrs['t0']=0
        f[filename].attrs['phi0']=0
        f[filename].attrs['th0']=np.pi/2.0
        f[filename].attrs['ur0']=0
        f[filename].attrs['uth0']=0
        for i in f[filename].attrs:
            
            print(i+": %f"%(f[filename].attrs[i]))
f.close()

5.0 0.9
skip unstable orbit
5.0 0.91
skip unstable orbit
5.0 0.92
1
derivative finished
complex128
/Kerr/tmp1
r0: 5.000000
E: 0.920000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
5.0 0.93
2
derivative finished
complex128
/Kerr/tmp2
r0: 5.000000
E: 0.930000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
5.0 0.94
3
derivative finished
complex128
/Kerr/tmp3
r0: 5.000000
E: 0.940000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
5.0 0.95
4
derivative finished
complex128
/Kerr/tmp4
r0: 5.000000
E: 0.950000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
5.0 0.96
5
derivative finished
complex128
/Kerr/tmp5
r0: 5.000000
E: 0.960000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
5.0 0.97
6
derivative finished
complex128
/Kerr/tmp6
r0: 5.000000
E: 0.970000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th

47
derivative finished
complex128
/Kerr/tmp47
r0: 8.000000
E: 0.970000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
8.0 0.98
48
derivative finished
complex128
/Kerr/tmp48
r0: 8.000000
E: 0.980000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
8.0 0.99
49
derivative finished
complex128
/Kerr/tmp49
r0: 8.000000
E: 0.990000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
8.5 0.9
skip unstable orbit
8.5 0.91
skip unstable orbit
8.5 0.92
skip unstable orbit
8.5 0.93
skip unstable orbit
8.5 0.94
50
derivative finished
complex128
/Kerr/tmp50
r0: 8.500000
E: 0.940000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
8.5 0.95
51
derivative finished
complex128
/Kerr/tmp51
r0: 8.500000
E: 0.950000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
8.5 0.96
52
derivative finished
complex128
/Kerr/tmp52
r0: 8.500

91
derivative finished
complex128
/Kerr/tmp91
r0: 11.500000
E: 0.990000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
12.0 0.9
trace_spin0.500000_d10.000000_r012.000000_E0.900000.dat  does not exist
92
derivative finished
complex128
/Kerr/tmp92
r0: 12.000000
E: 0.900000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
12.0 0.91
trace_spin0.500000_d10.000000_r012.000000_E0.910000.dat  does not exist
93
derivative finished
complex128
/Kerr/tmp93
r0: 12.000000
E: 0.910000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
12.0 0.92
skip unstable orbit
12.0 0.93
skip unstable orbit
12.0 0.94
skip unstable orbit
12.0 0.95
94
derivative finished
complex128
/Kerr/tmp94
r0: 12.000000
E: 0.950000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
12.0 0.96
95
derivative finished
complex128
/Kerr/tmp95
r0: 12.000000
E: 0.960000
spin: 0.500000
t0: 0.00

derivative finished
complex128
/Kerr/tmp131
r0: 14.500000
E: 0.920000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
14.5 0.93
skip unstable orbit
14.5 0.94
skip unstable orbit
14.5 0.95
skip unstable orbit
14.5 0.96
132
derivative finished
complex128
/Kerr/tmp132
r0: 14.500000
E: 0.960000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
14.5 0.97
133
derivative finished
complex128
/Kerr/tmp133
r0: 14.500000
E: 0.970000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
14.5 0.98
134
derivative finished
complex128
/Kerr/tmp134
r0: 14.500000
E: 0.980000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
14.5 0.99
135
derivative finished
complex128
/Kerr/tmp135
r0: 14.500000
E: 0.990000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
15.0 0.9
trace_spin0.500000_d10.000000_r015.000000_E0.900000.dat  does not

derivative finished
complex128
/Kerr/tmp170
r0: 17.000000
E: 0.920000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
17.0 0.93
trace_spin0.500000_d10.000000_r017.000000_E0.930000.dat  does not exist
171
derivative finished
complex128
/Kerr/tmp171
r0: 17.000000
E: 0.930000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
17.0 0.94
skip unstable orbit
17.0 0.95
skip unstable orbit
17.0 0.96
172
derivative finished
complex128
/Kerr/tmp172
r0: 17.000000
E: 0.960000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
17.0 0.97
173
derivative finished
complex128
/Kerr/tmp173
r0: 17.000000
E: 0.970000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
17.0 0.98
174
derivative finished
complex128
/Kerr/tmp174
r0: 17.000000
E: 0.980000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
17.0 0.99
175
derivative finishe

209
derivative finished
complex128
/Kerr/tmp209
r0: 19.000000
E: 0.970000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
19.0 0.98
210
derivative finished
complex128
/Kerr/tmp210
r0: 19.000000
E: 0.980000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
19.0 0.99
211
derivative finished
complex128
/Kerr/tmp211
r0: 19.000000
E: 0.990000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
19.5 0.9
trace_spin0.500000_d10.000000_r019.500000_E0.900000.dat  does not exist
212
derivative finished
complex128
/Kerr/tmp212
r0: 19.500000
E: 0.900000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
19.5 0.91
trace_spin0.500000_d10.000000_r019.500000_E0.910000.dat  does not exist
213
derivative finished
complex128
/Kerr/tmp213
r0: 19.500000
E: 0.910000
spin: 0.500000
t0: 0.000000
phi0: 0.000000
th0: 1.570796
ur0: 0.000000
uth0: 0.000000
19.5 0.92
trace_s

In [104]:
for key in f.keys():
    print(key)    
    print(f[key].name)
    g=f[key]
    for key2 in g.keys():
        print(key2)    
        print(g[key2].name)
    
f.close()

ValueError: Not a location id (Invalid object id)

In [110]:
for name in f:
    print(name)


ValueError: Not a location id (Invalid object id)

In [112]:
def printname(name):
    print(name)

f=h5py.File("mytemplate.hdf5","w") 
f.visit(printname)


Kerr
Kerr/tmp1
Kerr/tmp3


In [46]:
f["/bar1/dset1/d1"].attrs['at2']=10

In [75]:
for i in f["/Kerr/tmp1"].attrs:
    print(i)
    print(f["/Kerr/tmp1"].attrs[i])

r0
20
E
0.968693
spin
0.5
t0
0
phi0
0
th0
1.57079632679
ur0
0
uth0
0


In [49]:
f["/bar1/dset1/d1"].shape

(100001,)

In [115]:

f=h5py.File("mytemplate.hdf5","r+") 
f.close()

In [2]:
print("1"+"%.6f"%(4))

14.000000
